In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import os
import sys
import csv
import pandas as pd
sys.path.append('../python')
import medwatch as mw

In [37]:
# f = open('../datasets/keywords_covid.csv', 'r')
# reader = csv.reader(f, delimiter = '\n')
# kws = list(reader)
# keywords = []
# for keyword in kws:
#     keywords.append(keyword[0])
# print(keywords)

In [6]:
keywords_csv = '../datasets/keywords_covid.csv'
keywords = mw.load_keywords_csv(keywords_csv)
print(keywords)

['clinical trial', 'phase I', 'phase 1', 'phase ii', 'phase 2', 'phase 1/2', 'phase iii', 'phase 3', 'late stage', 'federal', 'government', 'funding', 'operation warp speed', 'who health organization', 'WHO', 'FDA', 'doses', 'manufacturing', 'partner', 'partnership', 'vaccine', 'treatment', 'immune', 'immunity', 'antibody', 'antibodies', 'safety', 'adverse', 'reaction', 'beneficial']


In [25]:
path_data = '../datasets/WHO-covid19-landscape-07_06_20-update.csv'
df = pd.read_csv(path_data)
developers = df['Developer'].tolist()
companies = []


# GET LIST OF COMPANIES
# For each row, split to individual organizations
for developer in developers:
    if mw.is_na(developer):
        continue
        
    organizations = developer.split('/')
    
    # For each organization, clean up formatting and add to companies list if not academia
    for organization in organizations:
        organization = organization.replace('\n', ' ')
        if not mw.is_academia(organization):
            companies.append(organization)

companies = list(set(companies)) # removes duplicates
            
symbols = []
exchanges = []
ynames = []
is_us = []

for company in companies:
    sym, exch, yco, usa = mw.check_usa_mkts(mw.get_company_info(company))
    symbols.append(sym)
    exchanges.append(exch)
    ynames.append(yco)
    is_us.append(usa)

30
30


In [33]:
print(companies)
print(symbols)
print(is_us)
pd_data.get_quote_yahoo('TKBIF')['marketCap']

['Medicago Inc.', 'AstraZeneca', 'Cadila Healthcare Limited', 'Beijing Institute of Biotechnology', ' Takara Bio', 'Wuhan Institute of Biological Products', 'Beijing Institute of Biological Products', 'Sinovac', 'Gamaleya Research Institute', 'Moderna', ' Université Laval', 'Walvax Biotech.', 'Inovio Pharmaceuticals', 'BioNTech', 'NIAID', 'GSK', 'Vaxine Pty Ltd', 'Medytox', 'Clover Biopharmaceuticals Inc.', 'CanSino Biological Inc.', 'Curevac', 'Pfizer', 'Sinopharm', 'Anhui Zhifei Longcom Biopharmaceutical', 'Dynavax', 'Novavax', ' AnGes', ' International Vaccine Institute', 'Fosun Pharma', 'Genexine Consortium']
['n/a', 'AZN', 'n/a', 'n/a', 'TKBIF', 'n/a', 'n/a', 'SVA', 'n/a', 'MRNA', 'n/a', '300142.SZ', 'INO', 'BNTX', 'n/a', 'GSK', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'PFE', 'SHTDY', 'n/a', 'DVAX', 'NVAX', 'AMGXF', 'n/a', 'SFOSF', 'n/a']
['n/a', 'Y', 'n/a', 'n/a', 'N', 'n/a', 'n/a', 'Y', 'n/a', 'Y', 'n/a', 'N', 'Y', 'Y', 'n/a', 'Y', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'Y', 'N', 'n/a', '

NameError: name 'pd_data' is not defined

In [40]:
marketcaps = []
sizes = []
urls = []
urls_pr = []

for sym, co in zip(symbols, companies):
    if sym == 'n/a':
        print(f'Skipping {co}\n')
        marketcaps.append('n/a')
        sizes.append('n/a')
        urls.append('n/a')
        urls_pr.append('n/a')
        continue

    print(f'Checking {co} [{sym}]')
    marketcap = mw.get_market_cap(sym)
    size = mw.id_company_size(marketcap)
    url = mw.get_company_url(sym)
    url_pr = mw.get_press_release_page(url)
    
    marketcaps.append(marketcap)
    sizes.append(size)
    urls.append(url)
    urls_pr.append(url_pr)

print('Search complete')


Skipping Medicago Inc.

Checking AstraZeneca [AZN]
Searching for press release page on http://www.astrazeneca.com
Found: ['https://www.astrazeneca.com/media-centre/press-releases.html']
-------------------------------------

Skipping Cadila Healthcare Limited

Skipping Beijing Institute of Biotechnology

Checking  Takara Bio [TKBIF]
Market cap for TKBIF not found.
Market cap not defined.
Searching for press release page on http://www.takara-bio.com
Found: ['http://www.takara-bio.com/release/']
-------------------------------------

Skipping Wuhan Institute of Biological Products

Skipping Beijing Institute of Biological Products

Checking Sinovac [SVA]
Searching for press release page on http://www.sinovacbio.com
Found: ['http://www.sinovacbio.com/?optionid=754']
-------------------------------------

Skipping Gamaleya Research Institute

Checking Moderna [MRNA]
Searching for press release page on http://www.modernatx.com
Found: ['https://investors.modernatx.com/news-releases/']
------

In [42]:
df = pd.DataFrame({'Company': companies, 
                   'Yahoo Listed Co.': ynames, 
                   'Symbol': symbols, 
                   'Exchange': exchanges, 
                   'Market Cap': marketcaps, 
                   'Company Size': sizes, 
                   'Is American': is_us,
                   'Home URL': urls,
                   'Press Release URL': urls_pr})
df.to_csv('../datasets/updated_company_list.csv', index = False)